In [ ]:
%load_ext autoreload
%autoreload 2
import pandas as pd
from src.distributions_check import check_feature_distributions_by_stat_test, check_feature_distributions_by_model
from src.datasets import sample_features, TorchDataset, NumpyDataset

In [ ]:
path_to_data = 'data/criteo/50/train.tsv'
numpy_dataset = NumpyDataset(path_to_data)
numpy_dataset.data

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import pandas as pd
import numpy as np


class LinearModel(nn.Module):
    """
    Простая линейная модель с одним слоем. Используется для классификации.
    """
    def __init__(self, input_dim):
        super(LinearModel, self).__init__()
        self.linear = nn.Linear(input_dim, 1)  # Линейный слой: вход -> 1 выход (для бинарной классификации)
        self.sigmoid = nn.Sigmoid()  # Сигмоидная функция активации для выхода от 0 до 1

    def forward(self, x):
        x = self.linear(x)
        x = self.sigmoid(x)
        return x


def train_model(train_loader, model, criterion, optimizer, num_epochs=10):
    """
    Функция для обучения модели.
    """
    model.train()  # Переводим модель в режим обучения
    for epoch in range(num_epochs):
        running_loss = 0.0
        for inputs, labels, _ in train_loader:
            # Перемещаем данные на устройство (CPU/GPU)
            inputs, labels = inputs.to(device), labels.to(device)

            # Обнуляем градиенты
            optimizer.zero_grad()

            # Прямой проход
            outputs = model(inputs).squeeze()
            loss = criterion(outputs, labels)

            # Обратный проход (backpropagation)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)

        epoch_loss = running_loss / len(train_loader.dataset)
        print(f"Эпоха [{epoch + 1}/{num_epochs}], Потеря: {epoch_loss:.4f}")


def evaluate_model(test_loader, model):
    """
    Функция для оценки модели после обучения.
    """
    model.eval()  # Переводим модель в режим оценки
    y_true = []
    y_pred = []

    with torch.no_grad():
        for inputs, labels, _ in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs).squeeze()
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(outputs.cpu().numpy())

    # Переводим предсказания в бинарный формат (0 или 1) на основании 0.5 как порога.
    y_pred = np.array(y_pred) > 0.5
    accuracy = np.mean(y_pred == y_true)
    print(f"Точность: {accuracy:.4f}")


if __name__ == '__main__':
    # Данные
    train_path = 'data/criteo/100/train.tsv'
    test_path = 'data/criteo/100/test.tsv'

    # Загружаем данные как PyTorch Dataset
    train_dataset = TorchDataset(train_path)
    test_dataset = TorchDataset(test_path)

    # DataLoader для итерирования по данным (батчи)
    train_loader = DataLoader(train_dataset[:1000], batch_size=32, shuffle=True)
    test_loader = DataLoader(test_dataset[:1000], batch_size=32, shuffle=False)

    # Параметры модели
    input_dim = train_dataset.data.shape[1]  # Число фичей (-1 потому что последняя колонка — таргет)
    model = LinearModel(input_dim=input_dim)

    # Устройство (CPU или GPU)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    # Функция потерь и оптимизатор
    criterion = nn.BCELoss()  # Binary Cross Entropy Loss для бинарной классификации
    optimizer = optim.Adam(model.parameters(), lr=0.01)

    # Обучение модели
    train_model(train_loader, model, criterion, optimizer, num_epochs=10)

    # Оценка модели
    evaluate_model(test_loader, model)